# **Topic Modeling and Search with Top2Vec :** 
# **Adventure Cove**

# **1. Import and Setup**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/nlp')

## **a) To install Top2Vec library and pre-trained BERT sentence transformer options:**

In [ ]:
# !pip install top2vec[sentence_transformers]
!pip install pynndescent


## **b) Import Libraries**

In [ ]:
from top2vec import Top2Vec 

import pandas as pd
import numpy as np

# **2. Import Dataset**

In [ ]:
df = pd.read_excel("/content/gdrive/MyDrive/data/clean/formatted_reviews.xlsx")
df.sample(4)

,date,source,attraction,reviews,rating
995,2019.0,google_reviews,uss,tempat yang keren buat liburan,8.0
3127,2020.0,traveloka,uss,baguss banget sayang antrinya saja,10.0
2700,2019.0,traveloka,uss,pertama kali ke singapore ala backpacker enjoy...,9.0
1653,2020.0,google_reviews,uss,recomended show transformer jurasic park sama ...,10.0


In [ ]:
df['date'] = df['date'].astype(int)

In [ ]:
cove_df = df[df['attraction'] == 'adventure_cove']
cove_df = cove_df.sort_values(by=['date','rating'],ascending=[False,False])
# resetting index
cove_df = cove_df.reset_index(drop=True)
cove_df.tail(4)

,date,source,attraction,reviews,rating
68,2018,klook,adventure_cove,pengalaman yang tidak terlupakan dengan menyel...,8.0
69,2018,google_reviews,adventure_cove,tempat yang bagus untuk rekreasi keluarga muda...,6.0
70,2017,klook,adventure_cove,pengalaman menyenangkan beli tiket melalui klo...,10.0
71,2017,traveloka,adventure_cove,pengalaman yang sangat menyenangkan,8.0


In [ ]:
cove_docs = cove_df.loc[:, "reviews"].astype(str).values.tolist()
cove_docs[:7]

['pilihan luar biasa untuk menghabiskan hari yang menyenangkan bersama keluarga dan teman kami mengalami hari yang luar biasa dikunjungi pada 15 mei 2022 bersama teman dan keluarga semua wahana dibuka kami mencapai pukul 10 dan karenanya untuk beberapa perjalanan pertama tidak ada waktu tunggu dan kami melakukan perjalanan beberapa kali hanya dalam 30 menit setelah itu sebagian besar memiliki waktu tunggu dan antrian sekitar 15 45 menit kami mengunjungi pada akhir pekan yang panjang sehingga lebih banyak orang tapi penantian itu sepadan dengan roket riptide wahana air lainnya juga bagus untuk beberapa wahana anda perlu membawa tabung karet dari bawah dan membawa sepanjang jalan di bahu anda menaiki tangga tidak suka bagian ini tidak semua orang cocok untuk melakukan ini park harus membuat beberapa pengaturan untuk itu kolam ombak sangat bagus mereka yang tahu berenang dan pernah melakukan snorkeling sebelumnya itu adalah pengalaman yang luar biasa untuk melihat karang dan kehidupan lau

# **3. Create and Train N-Gram model (Bi-Gram and Tri-Gram)**
* Top2Vec uses Gensim **simple_preprocess** to do tokenization. 
* To create Trigrams and Bigrams, pass a customized tokenizer into Top2Vec tokenizer parameter. 
* Use Gensim to train this N-gram model. Documents passed to the Phrases function to inspect for N-grams text corpus.

In [ ]:
import gensim
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import strip_tags

# Build the bigram and trigram models 
sentence_stream = [doc.split(" ") for doc in cove_docs]
bigram = Phrases(sentence_stream, min_count=5, threshold=5, delimiter=b' ')
trigram = Phrases(bigram[sentence_stream], threshold=5, delimiter=b' ')

bigram_phraser = Phraser(bigram)
trigram_phraser = Phraser(trigram)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
# Create Bi-Grams
def bi_gram(doc):
    sentence_stream = simple_preprocess(strip_tags(str(doc)), deacc=True)
    return bigram_phraser[sentence_stream] 


# Create Bi-Grams and Tri-Grams
def tri_gram(doc):
    sentence_stream = simple_preprocess(strip_tags(str(doc)), deacc=True)
    gen_bigram = bigram_phraser[sentence_stream]
    gen_trigram = trigram_phraser[bigram_phraser[sentence_stream]]
    return gen_trigram

# **4. Train Top2Vec Model**

### **Parameters**:
* **Documents**: Input corpus, should be a list of strings.
* **Min_count**: (Optional, default 50) Ignores all words with total frequency lower than this. For smaller corpora a smaller min_count will be necessary.
* **Embedding_model** (string or callable) – The valid string options are: doc2vec , universal-sentence-encoder , universal-sentence-encoder-multilingual,distiluse-base-multilingual-cased , all-MiniLM-L6-v2 , paraphrase-multilingual-MiniLM-L12-v2 
* **tokenizer** (callable (Optional, default None)) – Override the default tokenization method. If None then gensim.utils.simple_preprocess will be used. ***Tokenizer must take a document and return a list of tokens***.

In [ ]:
# FORM UNI-GRAMS, BI-GRAMS WITH BI-GRAM MODEL 

topicMODEL_bi = Top2Vec(documents= cove_docs, min_count=5, embedding_model = 'distiluse-base-multilingual-cased', tokenizer= bi_gram)  

2022-06-25 08:37:42,671 - top2vec - INFO - Pre-processing documents for training
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
2022-06-25 08:37:42,721 - top2vec - INFO - Downloading distiluse-base-multilingual-cased model


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/528 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

2022-06-25 08:38:45,267 - top2vec - INFO - Creating joint document/word embedding
2022-06-25 08:38:54,777 - top2vec - INFO - Creating lower dimension embedding of documents
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2022-06-25 08:39:08,961 - top2vec - INFO - Finding dense areas of documents
2022-06-25 08:39:09,015 - top2vec - INFO - Finding topics


In [ ]:
# FORM UNI-GRAMS, BI-GRAMS, TRI-GRAMS WITH TRI-GRAM MODEL 
 
topicMODEL_tri = Top2Vec(documents= cove_docs, min_count=5, embedding_model = 'distiluse-base-multilingual-cased', tokenizer=tri_gram)  

2022-06-25 08:39:12,726 - top2vec - INFO - Pre-processing documents for training
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
2022-06-25 08:39:12,790 - top2vec - INFO - Downloading distiluse-base-multilingual-cased model
2022-06-25 08:39:21,976 - top2vec - INFO - Creating joint document/word embedding
2022-06-25 08:39:36,825 - top2vec - INFO - Creating lower dimension embedding of documents
2022-06-25 08:39:40,726 - top2vec - INFO - Finding dense areas of documents
2022-06-25 08:39:40,739 - top2vec - INFO - Finding topics


In [ ]:
# SAVE MODELS FOR FUTURE USE

topicMODEL_bi.save("/content/gdrive/MyDrive/data/models/bigram_cove_june25")

topicMODEL_tri.save("/content/gdrive/MyDrive/data/models/trigram_cove_june25")

In [ ]:
# LOAD MODEL FROM SAVED MODELS

topicMODEL_bi = Top2Vec.load("/content/gdrive/MyDrive/data/models/bigram_cable_june25")

topicMODEL_tri = Top2Vec.load("/content/gdrive/MyDrive/data/models/trigram_cable_june25")

# **5. Perform Topic Reduction** 

* Reduce the number of topics discovered by Top2Vec.
* The most representative topics of the corpus will be found, by iteratively merging each smallest topic to the most similar topic until num_topics is reached. 
* Get the hierarchy of reduced topics. The mapping of each original topic to the reduced topics is returned.

In [ ]:
# BI-GRAM MODEL TOPIC REDUCTION
topicMODEL_bi.hierarchical_topic_reduction(num_topics=10) 

topicMODEL_bi.get_topic_hierarchy()

In [ ]:
# TRI-GRAM MODEL TOPIC REDUCTION 
topicMODEL_tri.hierarchical_topic_reduction(num_topics=10) 

topicMODEL_tri.get_topic_hierarchy()

# **6. Explore Discovered Topics**

## **6a) Get Number of Topics**
This will return the number of topics that Top2Vec has found in the data.

In [ ]:
# Show the Total Number of Topics 

print(f"Total Number of Topics generated (Topic Reduction Applied): \n")
print(f"Bi-gram Model : {topicMODEL_bi.get_num_topics(reduced=False)}")
print(f"Tri-gram Model : {topicMODEL_tri.get_num_topics(reduced=False)}")

Total Number of Topics generated (Topic Reduction Applied): 

Bi-gram Model : 2
Tri-gram Model : 2


## **6b) Get Topic Sizes**
This will return the number of documents most similar to each topic. Topics are in decreasing order of size.

Returns:
* topic_sizes: The number of documents most similar to each topic.
* topic_nums: The unique index of every topic will be returned.

### **6b.1. Bi-Gram and Tri-Gram Model Topic Sizes**

In [ ]:
print("BI-GRAM MODEL where N = 1 and 2")
print("================================\n")
topic_sizes_bi, topic_nums_bi = topicMODEL_bi.get_topic_sizes(reduced=False) 

print(f"Unique index numbers of every topic: {topic_nums_bi} ")
print()
print(f"Number of documents for each unique topic: {topic_sizes_bi} \n")

print("TRI-GRAM MODEL where N = 1, 2 and 3")
print("====================================\n")
topic_sizes_tri, topic_nums_tri = topicMODEL_tri.get_topic_sizes(reduced=False) 
print(f"Unique index numbers of every topic: {topic_nums_tri} ")
print()
print(f"Number of documents for each unique topic: {topic_sizes_tri} ") 

BI-GRAM MODEL where N = 1 and 2

Unique index numbers of every topic: [0 1] 

Number of documents for each unique topic: [46 26] 

TRI-GRAM MODEL where N = 1, 2 and 3

Unique index numbers of every topic: [0 1] 

Number of documents for each unique topic: [40 32] 


## **6c) Get Topics**
This will return the topics in decreasing size.

Returns:

* topic_words: For each topic the top 50 words are returned, in decreasing order of semantic similarity to topic.

* word_scores: For each topic the cosine similarity scores (in decreasing order) of the top 50 words to the topic are returned.

* topic_nums: The unique index of every topic will be returned.

### **c.1.  BI-GRAM Model Topics**

In [ ]:
print(f"TOPICS FOR BI-GRAM MODEL: \n")
topic_words_bi, word_scores_bi, topic_nums_bi = topicMODEL_bi.get_topics(topicMODEL_bi.get_num_topics(reduced=False))

for count,(words_bi, scores_bi, num_bi) in enumerate(zip(topic_words_bi, word_scores_bi, topic_nums_bi)):
    print(f"{count+1}. Topic {num_bi+1}") 
    print(f"Topic Words : \n{words_bi} \n")
    # print(f"Cosine Similarity :\n{scores_bi}\n")

TOPICS FOR BI-GRAM MODEL: 

1. Topic 1
Topic Words : 
['kolam ombak' 'kolam' 'snorkeling' 'sangat menyenangkan' 'menyenangkan'
 'taman' 'snorkling' 'ikan' 'menikmati' 'sangat' 'bagus' 'cukup' 'panjang'
 'beberapa wahana' 'suka' 'besar' 'mudah' 'tidak' 'tetapi' 'banyak'
 'untuk' 'karena' 'beberapa' 'pengalaman' 'semua' 'anakanak' 'sekali'
 'jadi' 'lain' 'hanya' 'yang' 'adalah' 'bisa' 'menit' 'itu' 'bersih' 'ini'
 'ke' 'di' 'dan' 'dengan' 'tapi' 'juga' 'tersedia' 'paling' 'tiket'
 'permainan' 'pagi' 'salah satu' 'anak'] 

2. Topic 2
Topic Words : 
['sangat menyenangkan' 'menyenangkan' 'menikmati' 'bagus' 'keluarga'
 'bersih' 'permainan' 'anakanak' 'anak' 'beberapa wahana' 'pengalaman'
 'besar' 'suka' 'tempat' 'untuk' 'sore' 'bersama' 'wahana' 'loker' 'taman'
 'perjalanan' 'sangat' 'kolam ombak' 'karena' 'dengan' 'paling' 'cukup'
 'kolam' 'banyak' 'mudah' 'tersedia' 'dan' 'hari' 'biasa' 'tiket' 'klook'
 'beberapa' 'lain' 'pagi' 'snorkling' 'snorkeling' 'menjadi' 'pada' 'yang'
 'di' 'panja

### **c.2. Tri-Gram Model Topics**

In [ ]:
print(f"TOPICS FOR TRI-GRAM MODEL : \n")
topic_words_tri, word_scores_tri, topic_nums_tri = topicMODEL_tri.get_topics(topicMODEL_tri.get_num_topics(reduced=False))

for count, (words_tri, scores_tri, num_tri) in enumerate(zip(topic_words_tri, word_scores_tri, topic_nums_tri)):
    print(f"{count+1}. Topic {num_tri+1}") 
    print(f"Topic Words : \n{words_tri}\n")
    # print(f"Cosine Similarity :\n{scores_tri}\n")

TOPICS FOR TRI-GRAM MODEL : 

1. Topic 1
Topic Words : 
['kolam ombak' 'kolam' 'snorkeling' 'sangat menyenangkan' 'menyenangkan'
 'taman' 'snorkling' 'ikan' 'menikmati' 'sangat' 'cukup' 'bagus' 'panjang'
 'suka' 'beberapa wahana' 'tidak' 'mudah' 'semua' 'pengalaman' 'banyak'
 'untuk' 'tetapi' 'sekali' 'karena' 'besar' 'hanya' 'adalah' 'beberapa'
 'itu' 'jadi' 'yang' 'lain' 'bisa' 'ini' 'ke' 'bersih' 'di' 'anakanak'
 'dan' 'menit' 'tersedia' 'dengan' 'juga' 'permainan' 'tapi' 'paling'
 'tiket' 'ada' 'akan' 'salah satu']

2. Topic 2
Topic Words : 
['sangat menyenangkan' 'menyenangkan' 'menikmati' 'bagus' 'keluarga'
 'pengalaman' 'anakanak' 'anak' 'kolam ombak' 'snorkeling' 'kolam'
 'permainan' 'suka' 'bersih' 'beberapa wahana' 'besar' 'sore' 'snorkling'
 'wahana' 'untuk' 'bersama' 'perjalanan' 'taman' 'paling' 'tempat'
 'dengan' 'karena' 'loker' 'tersedia' 'pagi' 'hari' 'dan' 'air' 'sangat'
 'pada' 'yang' 'biasa' 'lain' 'ikan' 'klook' 'tiket' 'panjang' 'di' 'jadi'
 'nya' 'mudah' 'menjadi

## **6d) Search Documents by Topic**
We are going to search by topic; for example Topic 3. 

Returns:

* documents:  The documents in a list, the most similar are first.
* doc_scores:  Semantic similarity of document to topic. The cosine similarity of the document and topic vector.
* doc_ids:  Unique ids of documents. If ids were not given, the index of document in the original corpus.

For each of the returned documents we are going to print its content, score and document number.

### **d.1. Search Documents By Topic Number Using Bi-Gram Model**

In [ ]:
# SEARCH DOCUMENTS BY TOPIC NUMBER USING BI-GRAM MODEL 
topic_num_bi = 0
documents_bi, document_scores_bi, document_ids_bi = topicMODEL_bi.search_documents_by_topic(topic_num= topic_num_bi , num_docs= 5, reduced=False) 

print(f"TOPIC NUMBER: {topic_num_bi}\n")
for count, (doc_bi, docscore_bi, doc_id_bi) in enumerate(zip(documents_bi, document_scores_bi, document_ids_bi)):
    print(f"{count}. Document: {doc_id_bi},  Score: {docscore_bi}")
    print("-----------")
    print(doc_bi)
    print("-----------\n")

TOPIC NUMBER: 0

0. Document: 50,  Score: 0.6376216411590576
-----------
splish splash dan semua ikan itu itu adalah salah satu kunjungan pertama kami ke taman air internasional sebagai liburan orang dewasa dan kami sangat terkejutruang ganti rapi dan merasa amanberbagai kegiatan air sangat menyenangkan kami menikmati kolam ombak besar dan setelah dihantam sedikit tentang kami pergi di sungai malas kami melakukan beberapa slide besar dan menikmati area snorkelingkami tinggal di pulau sentosa dengan tiket masuk yang termasuk dalam pemesanan hotel kami dan menikmati menghabiskan sore di taman air
-----------

1. Document: 1,  Score: 0.6246395111083984
-----------
bagus tempat yang bagus untuk anakanak dengan cuaca lembab seperti itu kami pergi ke sana pada hari sabtu pagi dan hujan mulai turun itu menyenangkan untuk anakanakloker mahal untuk loker kecil yang dapat memuat ransel ukuran notebook dan sepatu 3 pars berharga 10 dan loker besar dapat memuat ransel besar untuk hiking seharga 20

### **d.2. Search Documents By Topic Number Using Tri-Gram Model**

In [ ]:
# SEARCH DOCUMENTS BY TOPIC NUMBER USING TRI-GRAM MODEL 
topic_num_tri = 1
documents_tri, document_scores_tri, document_ids_tri = topicMODEL_tri.search_documents_by_topic(topic_num= topic_num_tri, num_docs= 5, reduced=False) 
print(f"TOPIC NUMBER: {topic_num_bi}\n")
for count, (doc_tri, docscore_tri, doc_id_tri) in enumerate(zip(documents_tri, document_scores_tri, document_ids_tri)):
    print(f"{count}. Document: {doc_id_tri},  Score: {docscore_tri}")
    print("-----------")
    print(doc_tri)
    print("-----------\n")

TOPIC NUMBER: 0

0. Document: 45,  Score: 0.4467974305152893
-----------
sangat menyenangkan tidak dapat dilupakan untuk anak kolamnya kids friendly
-----------

1. Document: 51,  Score: 0.43852555751800537
-----------
menyenangkan dan pengalaman seru anakanak senang
-----------

2. Document: 35,  Score: 0.4352504014968872
-----------
bagus dan menyenangkan cocok untuk anak2 dan anak muda
-----------

3. Document: 69,  Score: 0.41908884048461914
-----------
tempat yang bagus untuk rekreasi keluarga mudah dijangkau dan bersih serta menarik
-----------

4. Document: 24,  Score: 0.409264475107193
-----------
pengalaman water boom yang menyenangkan
-----------



# **7) Constructing Dataframe to Collate Information of Key Topics**

## **7a) BI-GRAM MODEL**

In [ ]:
# CONSTRUCT DATAFRAME OF BI-GRAM MODEL WITH KEY INFORMATION FOR EXPORT
# initialize data of lists.
bigram_table = {
              'topic number': [num_bi for num_bi in topic_nums_bi],
              'topic words': [words_bi for words_bi in topic_words_bi],
              'cosine similarity metrics': [scores_bi for scores_bi in word_scores_bi],
              'similar documents': [size_bi for size_bi in topic_sizes_bi],
               }
 
# Create DataFrame
bigram_df = pd.DataFrame(bigram_table)
bigram_df['topic number'] = bigram_df['topic number'].apply(lambda x: x + 1)

# print(bigram_df.head(2))
bigram_df.tail()

,topic number,topic words,cosine similarity metrics,similar documents
0,1,"[kolam ombak, kolam, snorkeling, sangat menyen...","[0.3060581, 0.24878022, 0.24188027, 0.20341243...",46
1,2,"[sangat menyenangkan, menyenangkan, menikmati,...","[0.44214204, 0.3500477, 0.3174553, 0.22942856,...",26


### **7a.1. Build Functions to Expand Column Features in Bi-Gram Dataframe**

In [ ]:
def get_document_index(topic_num_bi, num_doc):
    documents_bi, document_scores_bi, document_ids_bi = topicMODEL_bi.search_documents_by_topic(topic_num= topic_num_bi , num_docs= num_doc, reduced=False)
    document_index = [doc_id_bi for doc_id_bi in document_ids_bi]
    return document_index 

def get_document_scores(topic_num_bi, num_doc):
    documents_bi, document_scores_bi, document_ids_bi = topicMODEL_bi.search_documents_by_topic(topic_num= topic_num_bi , num_docs= num_doc, reduced=False)
    document_scores = [docscore_bi for docscore_bi in document_scores_bi]
    return document_scores


def get_documents(topic_num_bi, num_doc):
    documents_bi, document_scores_bi, document_ids_bi = topicMODEL_bi.search_documents_by_topic(topic_num= topic_num_bi , num_docs= num_doc, reduced=False)
    actual_document = [doc_bi for doc_bi in documents_bi]
    return actual_document


In [ ]:
bigram_df['document index'] = bigram_df['topic number'].apply(lambda x: get_document_index(x-1, bigram_df['similar documents'].values[x-1]))

bigram_df['document scores'] = bigram_df['topic number'].apply(lambda x: get_document_scores(x-1, bigram_df['similar documents'].values[x-1]))

bigram_df['actual reviews'] = bigram_df['topic number'].apply(lambda x: get_documents(x-1, bigram_df['similar documents'].values[x-1]))

bigram_df

,topic number,topic words,cosine similarity metrics,similar documents,document index,document scores,actual reviews
0,1,"[kolam ombak, kolam, snorkeling, sangat menyen...","[0.3060581, 0.24878022, 0.24188027, 0.20341243...",46,"[50, 1, 57, 27, 3, 14, 62, 65, 41, 44, 6, 28, ...","[0.63762164, 0.6246395, 0.61481625, 0.6124438,...",[splish splash dan semua ikan itu itu adalah s...
1,2,"[sangat menyenangkan, menyenangkan, menikmati,...","[0.44214204, 0.3500477, 0.3174553, 0.22942856,...",26,"[69, 39, 35, 54, 51, 47, 18, 31, 58, 64, 43, 2...","[0.4826601, 0.4352029, 0.43166375, 0.43162882,...",[tempat yang bagus untuk rekreasi keluarga mud...


In [ ]:
# for count, (index, review) in enumerate(zip(bigram_df['document index'][28], bigram_df['actual reviews'][28])):
#   print(f"{count}: {index} = {review})")

bigram_df.to_excel(f'/content/gdrive/MyDrive/data/export/bigram_df_cove.xlsx', index=False)

## **7b) TRI-GRAM MODEL**

In [ ]:
# CONSTRUCT DATAFRAME CONTAINING KEY INFORMATION FOR EXPORT
# initialize data of lists.
trigram_table = {
                'topic number': [num_tri for num_tri in topic_nums_tri],
                'topic words': [words_tri for words_tri in topic_words_tri],
                'cosine similarity metrics': [scores_tri for scores_tri in word_scores_tri],
                'similar documents': [size_tri for size_tri in topic_sizes_tri],
                 }

# Create DataFrame
trigram_df = pd.DataFrame(trigram_table)

trigram_df['topic number'] = trigram_df['topic number'].apply(lambda x: x + 1)
trigram_df.tail()

,topic number,topic words,cosine similarity metrics,similar documents
0,1,"[kolam ombak, kolam, snorkeling, sangat menyen...","[0.30160916, 0.24567278, 0.24310988, 0.1987978...",40
1,2,"[sangat menyenangkan, menyenangkan, menikmati,...","[0.4582212, 0.37582523, 0.34498256, 0.23756793...",32


### **7b.1. Build Functions to Expand Column Features in Tri-Gram Dataframe**

In [ ]:
def get_document_index_tri(topic_num_tri, num_docs):
    documents_tri, document_scores_tri, document_ids_tri = topicMODEL_tri.search_documents_by_topic(topic_num= topic_num_tri , num_docs= num_docs, reduced=False)
    document_index = [doc_id_tri for doc_id_tri in document_ids_tri]
    return document_index 

def get_document_scores_tri(topic_num_tri, num_docs):
    documents_tri, document_scores_tri, document_ids_tri = topicMODEL_tri.search_documents_by_topic(topic_num= topic_num_tri , num_docs= num_docs, reduced=False)
    document_scores = [docscore_tri for docscore_tri in document_scores_tri]
    return document_scores


def get_documents_tri(topic_num_tri, num_docs):
    documents_tri, document_scores_tri, document_ids_tri = topicMODEL_tri.search_documents_by_topic(topic_num= topic_num_tri , num_docs= num_docs, reduced=False)
    actual_document = [doc_tri for doc_tri in documents_tri]
    return actual_document


In [ ]:
trigram_df['document index'] = trigram_df['topic number'].apply(lambda x: get_document_index_tri(x-1, trigram_df['similar documents'].values[x-1]))

trigram_df['document scores'] = trigram_df['topic number'].apply(lambda x: get_document_scores_tri(x-1, trigram_df['similar documents'].values[x-1]))

trigram_df['actual reviews'] = trigram_df['topic number'].apply(lambda x: get_documents_tri(x-1, trigram_df['similar documents'].values[x-1]))

trigram_df

,topic number,topic words,cosine similarity metrics,similar documents,document index,document scores,actual reviews
0,1,"[kolam ombak, kolam, snorkeling, sangat menyen...","[0.30160916, 0.24567278, 0.24310988, 0.1987978...",40,"[50, 27, 1, 57, 14, 62, 3, 65, 41, 44, 15, 6, ...","[0.646231, 0.622467, 0.61615634, 0.6076138, 0....",[splish splash dan semua ikan itu itu adalah s...
1,2,"[sangat menyenangkan, menyenangkan, menikmati,...","[0.4582212, 0.37582523, 0.34498256, 0.23756793...",32,"[45, 51, 35, 69, 24, 67, 52, 12, 54, 39, 18, 4...","[0.44679743, 0.43852556, 0.4352504, 0.41908884...",[sangat menyenangkan tidak dapat dilupakan unt...


In [ ]:
# for count, (index, review) in enumerate(zip(trigram_df['document index'][0], trigram_df['actual reviews'][0])):
#   print(f"{count}: {index} = {review})")

trigram_df.to_excel(f'/content/gdrive/MyDrive/data/export/trigram_df_cove.xlsx', index=False)

# **8. Defining Topic Labels on Reviews** 
## **8a) User-Define Topic Labels** 
### **8a.1:  BI-GRAM Model** 

In [ ]:
# Create the dictionary containing the data of the new column
bigram_col_dict = { 'Topic 1': 'features visitors like ',                 'Topic 2': 'overall experience',
                  
                  }
                   
# Assign the values of the dictionary as the values of the new column
bigram_df['topic label'] = bigram_col_dict.values()
bigram_df

,topic number,topic words,cosine similarity metrics,similar documents,document index,document scores,actual reviews,topic label
0,1,"[kolam ombak, kolam, snorkeling, sangat menyen...","[0.3060581, 0.24878022, 0.24188027, 0.20341243...",46,"[50, 1, 57, 27, 3, 14, 62, 65, 41, 44, 6, 28, ...","[0.63762164, 0.6246395, 0.61481625, 0.6124438,...",[splish splash dan semua ikan itu itu adalah s...,features visitors like
1,2,"[sangat menyenangkan, menyenangkan, menikmati,...","[0.44214204, 0.3500477, 0.3174553, 0.22942856,...",26,"[69, 39, 35, 54, 51, 47, 18, 31, 58, 64, 43, 2...","[0.4826601, 0.4352029, 0.43166375, 0.43162882,...",[tempat yang bagus untuk rekreasi keluarga mud...,overall experience


In [ ]:
# CREATE COPY OF ORIGINAL DATAFRAME
cove_DF_BI = cove_df.copy()
cove_DF_BI.tail(3)

,date,source,attraction,reviews,rating
69,2018,google_reviews,adventure_cove,tempat yang bagus untuk rekreasi keluarga muda...,6.0
70,2017,klook,adventure_cove,pengalaman menyenangkan beli tiket melalui klo...,10.0
71,2017,traveloka,adventure_cove,pengalaman yang sangat menyenangkan,8.0


In [ ]:
print([i for i in range(topicMODEL_bi.get_num_topics(reduced=False))])

[0, 1]


In [ ]:
# FUNCTION TO CONNECT TOPIC LABELS TO INDEX LIST
def get_indexLIST(i):
      topic = [doc_ind for doc_ind in bigram_df['document index'][i]]
      return topic

In [ ]:
for i in range(topicMODEL_bi.get_num_topics(reduced=False)):
    cove_DF_BI.loc[get_indexLIST(i), 'topic label'] = bigram_df['topic label'][i]
    

In [ ]:
# FUNCTION TO CONNECT DOCUMENT COSINE SCORE TO INDEX LIST
def get_scores(i):
    score = [doc_score for doc_score in bigram_df['document scores'][i]]
    return score

In [ ]:
for i in range(topicMODEL_bi.get_num_topics(reduced=False)):
    cove_DF_BI.loc[get_indexLIST(i), 'cosine score review/topic'] = get_scores(i) 

In [ ]:
cove_DF_BI.reset_index(drop=True)
cove_DF_BI

,date,source,attraction,reviews,rating,topic label,cosine score review/topic
0,2022,tripadvisor,adventure_cove,pilihan luar biasa untuk menghabiskan hari yan...,10.0,features visitors like,0.445527
1,2022,tripadvisor,adventure_cove,bagus tempat yang bagus untuk anakanak dengan ...,8.0,features visitors like,0.624640
2,2022,tripadvisor,adventure_cove,hari keluarga yang sangat menyenangkan kami be...,8.0,features visitors like,0.434765
3,2022,tripadvisor,adventure_cove,air menyenangkan tetapi anda perlu kesabaran a...,6.0,features visitors like,0.573863
4,2022,tripadvisor,adventure_cove,jangan buang waktu dan uang anda buang waktu d...,2.0,features visitors like,0.447846
...,...,...,...,...,...,...,...
67,2018,klook,adventure_cove,bagus permainan airnya anak saya suka sekali k...,10.0,features visitors like,0.359012
68,2018,klook,adventure_cove,pengalaman yang tidak terlupakan dengan menyel...,8.0,features visitors like,0.475177
69,2018,google_reviews,adventure_cove,tempat yang bagus untuk rekreasi keluarga muda...,6.0,overall experience,0.482660
70,2017,klook,adventure_cove,pengalaman menyenangkan beli tiket melalui klo...,10.0,overall experience,0.262168


In [ ]:
cove_DF_BI.to_excel(f'/content/gdrive/MyDrive/data/export/cove_DF_BI.xlsx', index=False)

### **8a.2  TRI-GRAM Model** 

In [ ]:
# Create the dictionary containing the data of the new column
trigram_col_dict = { 'Topic 1': 'features visitors like ',                 'Topic 2': 'overall experience',
                  
                  }
   
# Assign the values of the dictionary as the values of the new column
trigram_df['topic label'] = trigram_col_dict.values()
trigram_df.tail(3)

,topic number,topic words,cosine similarity metrics,similar documents,document index,document scores,actual reviews,topic label
0,1,"[kolam ombak, kolam, snorkeling, sangat menyen...","[0.30160916, 0.24567278, 0.24310988, 0.1987978...",40,"[50, 27, 1, 57, 14, 62, 3, 65, 41, 44, 15, 6, ...","[0.646231, 0.622467, 0.61615634, 0.6076138, 0....",[splish splash dan semua ikan itu itu adalah s...,features visitors like
1,2,"[sangat menyenangkan, menyenangkan, menikmati,...","[0.4582212, 0.37582523, 0.34498256, 0.23756793...",32,"[45, 51, 35, 69, 24, 67, 52, 12, 54, 39, 18, 4...","[0.44679743, 0.43852556, 0.4352504, 0.41908884...",[sangat menyenangkan tidak dapat dilupakan unt...,overall experience


In [ ]:
# CREATE COPY OF ORIGINAL DATAFRAME
cove_DF_TRI = cove_df.copy()
cove_DF_TRI.tail(3)

,date,source,attraction,reviews,rating
69,2018,google_reviews,adventure_cove,tempat yang bagus untuk rekreasi keluarga muda...,6.0
70,2017,klook,adventure_cove,pengalaman menyenangkan beli tiket melalui klo...,10.0
71,2017,traveloka,adventure_cove,pengalaman yang sangat menyenangkan,8.0


In [ ]:
print([i for i in range(topicMODEL_tri.get_num_topics(reduced=False))])

[0, 1]


In [ ]:
# FUNCTION TO CONNECT TOPIC LABELS TO INDEX LIST
def get_indexLIST(i):
      topic = [doc_ind for doc_ind in trigram_df['document index'][i]]
      return topic

In [ ]:
for i in range(topicMODEL_tri.get_num_topics(reduced=False)):
    cove_DF_TRI.loc[get_indexLIST(i), 'topic_label'] = trigram_df['topic label'][i]

In [ ]:
# FUNCTION TO CONNECT DOCUMENT COSINE SCORE TO INDEX LIST
def get_scores(i):
    score = [doc_score for doc_score in trigram_df['document scores'][i]]
    return score

In [ ]:
for i in range(topicMODEL_tri.get_num_topics(reduced=False)):
    cove_DF_TRI.loc[get_indexLIST(i), 'cosine score review/topic'] = get_scores(i) 

In [ ]:
cove_DF_TRI.reset_index(drop=True)
cove_DF_TRI

,date,source,attraction,reviews,rating,topic_label,cosine score review/topic
0,2022,tripadvisor,adventure_cove,pilihan luar biasa untuk menghabiskan hari yan...,10.0,features visitors like,0.428040
1,2022,tripadvisor,adventure_cove,bagus tempat yang bagus untuk anakanak dengan ...,8.0,features visitors like,0.616156
2,2022,tripadvisor,adventure_cove,hari keluarga yang sangat menyenangkan kami be...,8.0,features visitors like,0.439044
3,2022,tripadvisor,adventure_cove,air menyenangkan tetapi anda perlu kesabaran a...,6.0,features visitors like,0.570076
4,2022,tripadvisor,adventure_cove,jangan buang waktu dan uang anda buang waktu d...,2.0,features visitors like,0.448254
...,...,...,...,...,...,...,...
67,2018,klook,adventure_cove,bagus permainan airnya anak saya suka sekali k...,10.0,overall experience,0.408352
68,2018,klook,adventure_cove,pengalaman yang tidak terlupakan dengan menyel...,8.0,features visitors like,0.482355
69,2018,google_reviews,adventure_cove,tempat yang bagus untuk rekreasi keluarga muda...,6.0,overall experience,0.419089
70,2017,klook,adventure_cove,pengalaman menyenangkan beli tiket melalui klo...,10.0,overall experience,0.285148


In [ ]:
cove_DF_TRI.isnull().values.any()

False

In [ ]:
cove_DF_TRI.to_excel(f'/content/gdrive/MyDrive/data/export/cove_DF_TRI.xlsx', index=False)

# **9.  Visualization**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
uss_topred = pd.read_excel("/content/gdrive/MyDrive/data/export/uss_DF_BIreduced.xlsx")
uss_topred.isnull().values.any()
uss_topred = uss_topred.loc[uss_topred['date'] != 2017]
uss_topred

In [ ]:
sns.color_palette("gist_rainbow")

In [ ]:
sns.set_palette("gist_rainbow")
sns.set_theme(style="ticks", font_scale=1.4)

In [ ]:
plt.figure(figsize=(18,10))

hue_order = [2022, 2021, 2020, 2019, 2018]
sns.histplot(data=uss_topred, y="topic_label", hue="date", hue_order=hue_order, multiple="stack", bins= 50, palette="Set1") 


In [ ]:
sns.color_palette('Set3')
plt.figure(figsize=(25,10))

hue_order = [2018, 2019, 2020,2021, 2022]
sns.histplot(data=uss_topred, x="topic_label", hue="date", hue_order=hue_order, multiple="dodge", bins=100, palette="Set1") 


In [ ]:
plt.figure(figsize=(16,11))

sns.countplot(data=uss_topred, y="topic_label", hue="date", dodge=False)